# 1. Вопросики

А можно в `os` вернуться назад в исходную директорию?

In [4]:
import os

In [5]:
os.getcwd()

'/Users/randomwalk/Desktop/Dev/dap_2023/sem08_write'

In [6]:
os.chdir('..')

In [7]:
os.getcwd()

'/Users/randomwalk/Desktop/Dev/dap_2023'

In [9]:
os.chdir('sem02_for')

In [10]:
os.getcwd()

'/Users/randomwalk/Desktop/Dev/dap_2023/sem02_for'

In [11]:
os.chdir('../sem08_write')

In [12]:
os.getcwd()

'/Users/randomwalk/Desktop/Dev/dap_2023/sem08_write'

# 2. Задачка про разметку


В файлике `markup.json` лежит разметка новостных статей на четыре категории. Разметку делали [в толоке.](https://toloka.yandex.ru/tasks)

```
{'inputValues': {'id': 1, 'version': 1},
 'workerId': 465,
 'taskId': 0,
 'outputValues': {'result': 'ok'}}
```

- в поле `inputValues` лежит `id` документа и его версия
- в поле `workerId` лежит `id` модератора, который делал разметку
- в поле `taskId` лежит номер таски, в которой модератору показалась новость (в одной таске могут быть несколько новостей)
- в поле `outputValues` лежат результаты разметки

In [57]:
import json

with open("markup.json", "r") as f:
    data = json.load(f)    
data[:5]

[{'inputValues': {'id': 1, 'version': 1},
  'workerId': 465,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 365,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 619,
  'taskId': 0,
  'outputValues': {'result': 'local'}},
 {'inputValues': {'id': 2, 'version': 1},
  'workerId': 759,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 2, 'version': 1},
  'workerId': 694,
  'taskId': 0,
  'outputValues': {'result': 'ok'}}]

### Задание 1 Ответьте по данным на следующие вопросы:

__а)__ Сколько всего уникальных новостей было размечено?

In [58]:
len({item['inputValues']['id'] for item in data})

1000

__б)__ Сколько всего есть уникальных категорий в поле `result`? Как они называются? 

In [59]:
len({item['outputValues']['result'] for item in data})

4

In [60]:
from collections import Counter
h = [item['outputValues']['result'] for item in data]
Counter(h)

Counter({'ok': 1448, 'local': 543, 'violence': 508, 'federal': 501})

__в)__ Сколько человек занимались разметкой?

In [61]:
len({item['workerId'] for item in data})

957

__г)__ Сколько новостей в среднем приходится на один `taskId`? 

In [62]:
h = [item['taskId'] for item in data]
cnt = Counter(h)
cnt = [item[1] for item in cnt.most_common()]
sum(cnt)/len(cnt)

8.982035928143713

__д)__ Правда ли, что все входные поля имеют структуру описанную выше? Напишите функцию, которая будет проверять консистентность входных данных. Если данные имеют другую структуру, функция должна вызывать исключение типа `ValueError`.

In [65]:
x = data[0]
x

{'inputValues': {'id': 1, 'version': 1},
 'workerId': 465,
 'taskId': 0,
 'outputValues': {'result': 'ok'}}

In [66]:
'inputValues' in x

True

In [67]:
x['outputValues'].get('result') is not None

True

In [68]:
len(x) == 4

True

In [69]:
isinstance(x.get('workerId'), int)

True

In [71]:
isinstance(4, int)

True

In [72]:
isinstance('4', int)

False

In [73]:
cond = [
    'inputValues' in x,
    x['outputValues'].get('result') is not None,
    len(x) == 4,
    isinstance(x.get('workerId'), int)
]

In [74]:
all(cond)

True

In [75]:
all([True, False, True])

False

In [76]:
all([True, True, True])

True

In [77]:
def check_format(x):
    cond = [
        'inputValues' in x,
        x['outputValues'].get('result') is not None,
        len(x) == 4,
        isinstance(x.get('workerId'), int)
    ]
    if all(cond):
        return x
    else:
        raise ValueError('Формат входных данных не валиден')

In [78]:
check_format(data[10])

{'inputValues': {'id': 4, 'version': 1},
 'workerId': 43,
 'taskId': 1,
 'outputValues': {'result': 'ok'}}

In [80]:
data = [check_format(item) for item in data]

In [81]:
def filter_not_valid_data(data):
    valid_data = []
    error_data = []
    for item in data:
        try:
            valid_data.append(check_format(item))
        except ValueError:
            error_data.append(item)
    return valid_data, error_data

In [82]:
valid_data, error_data = filter_not_valid_data(data)

In [83]:
error_data

[]

Дальше я все невалидные данные сохраняю в какую-нибудь отдельную таблицу и автоматически срабатывает алерт, который зовёт разработчика или аналитика посмотреть на них.

### Задание 2

Каждая новость даётся на разметку нескольким модераторам. Мы хотим агрегировать разметку до одного числа. Напишите функцию `aggregate`, которая будет делать это методом большинства. 

На выходе мы должны получить json вида: 

```
[
{'inputValues': {'id': 1, 'version': 1},
 'workers': [465, 10, 55],
 'taskIds': [0, 1, 18],
 'rawnumber': 3,
 'probability': 0.66,
 'algorithm': 'majority_vote',
 'outputValues': {'result': 'ok'}},
 
 {'inputValues': {'id': 2, 'version': 1},
 'workers': [463, 11],
 'taskIds': [0, 10],
 'rawnumber': 2,
 'probability': 1,
 'algorithm': 'majority_vote',
 'outputValues': {'result': 'local'}}
 ]
```

В поле `rawNumber` записано количество ответов модераторов. В поле `probability` доля ответов за больший класс (majority vote).

Если все три модератора проголосовали разные вердикты, выбирайте рандомный.

In [89]:
data[:3]

[{'inputValues': {'id': 1, 'version': 1},
  'workerId': 465,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 365,
  'taskId': 0,
  'outputValues': {'result': 'ok'}},
 {'inputValues': {'id': 1, 'version': 1},
  'workerId': 619,
  'taskId': 0,
  'outputValues': {'result': 'local'}}]

In [140]:
def aggregate(assigments):
    agg_ass = [ ]
    
    for item in assigments:
        item['key'] = (item['inputValues']['id'], item['inputValues']['version'])
        
    for k, item in groupby(sorted(assigments, key=at('key')), key=at('key')):
        aggregated = {}
        
        ret = at('workerId', 'taskId', 'inputValues', 'outputValues')
        workerIds, taskIds, inputvalues, outputvalues = zip(*map(ret, item))
        
        aggregated['workerIds'] = list(workerIds)
        aggregated['taskIds'] = list(taskIds)
        aggregated['inputValue'] = inputvalues[0]
        aggregated['outputValues'] = {'result': [at('result')(jtem) for jtem in outputvalues]}
        agg_ass.append(aggregated)

    return agg_ass

In [141]:
agg_data = aggregate(data)
agg_data[0]

{'workerIds': [465, 365, 619],
 'taskIds': [0, 0, 0],
 'inputValue': {'id': 1, 'version': 1},
 'outputValues': {'result': ['ok', 'ok', 'local']}}

In [120]:
item = [{'inputValues': {'id': 1, 'version': 1}, 'workerId': 465, 'taskId': 0,
         'outputValues': {'result': 'ok'}, 'key': (1, 1)}, 
        
        {'inputValues': {'id': 1, 'version': 1}, 'workerId': 365, 'taskId': 0,
         'outputValues': {'result': 'ok'}, 'key': (1, 1)}, 
        
        {'inputValues': {'id': 1, 'version': 1}, 'workerId': 619, 'taskId': 0, 
         'outputValues': {'result': 'local'}, 'key': (1, 1)}]

In [132]:
ret = at('workerId', 'taskId', 'inputValues', 'outputValues')
list(map(ret, item))

[(465, 0, {'id': 1, 'version': 1}, {'result': 'ok'}),
 (365, 0, {'id': 1, 'version': 1}, {'result': 'ok'}),
 (619, 0, {'id': 1, 'version': 1}, {'result': 'local'})]

In [133]:
list(zip(*map(ret, item)))

[(465, 365, 619),
 (0, 0, 0),
 ({'id': 1, 'version': 1}, {'id': 1, 'version': 1}, {'id': 1, 'version': 1}),
 ({'result': 'ok'}, {'result': 'ok'}, {'result': 'local'})]

In [136]:
taskIds, workerIds, inputvalues, outputvalues = zip(*map(ret, item))

In [137]:
outputvalues

({'result': 'ok'}, {'result': 'ok'}, {'result': 'local'})

### Задание 3

В данных иногда встречаются ситуации, когда три модератора сказали разные вердикты. Не очень понятно что делать с разметкой в таком случае. Давайте будем обрабатывать такие краевые ситуации с помощью какого-нибудь бизнес-правила.

Пусть на множестве наших вердикто задан порядок. Я хочу реализовать два правила: 

__1)__ брать самый жёсткий из выставленных

__2)__ брать взвешенный вердикт



In [147]:
def simple_mv(output):
    pass

In [143]:
WEIGHTS = {
    'ok': 0,
    'violence': 1,
    'local': 0.5,
    'federal': 0
}

AGG_METHOD = 'max'

In [144]:
def get_most_dangerous(output, w=WEIGHTS):
    pass

In [145]:
def get_average(output, w=WEIGHTS):
    pass

In [146]:
def get_verdict(output, w=WEIGHTS):
    
    mv_verd, prob = simple_mv(output)
    
    if prob > 0.5:
        return mv_verd
    else:
        if AGG_METHOD == 'max':
            raise NotImplementedError
        elif AGG_METHOD == 'avg':
            raise NotImplementedError
        elif AGG_METHOD == 'random':
            raise NotImplementedError
        else:
            raise NotImplementedError

# 3. Справочная информация

## 3.1 List comprehension

Я хочу посчитать сумму первых ста чисел.

In [24]:
n = 100
s = 0

for i in range(1, n + 1):
    s += i
s

5050

In [27]:
n = 100
sum([i for i in range(1, n + 1)])

5050

In [31]:
# Скобочки могут быть любыми 
{i for i in range(10)} # set

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [32]:
(i for i in range(10)) # генератор

<generator object <genexpr> at 0x106bb9f50>

## 3.2 Исключения

In [45]:
x = [7, 8, 9, 0, 4, -5]

for item in x:
    print(42 / item)

6.0
5.25
4.666666666666667


ZeroDivisionError: division by zero

In [46]:
x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / item)
    except:
        print('Я падаю!')

6.0
5.25
4.666666666666667
Я падаю!
10.5
-8.4


In [47]:
from math import log

x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / log(item + 1))
    except ZeroDivisionError:
        print('Не могу делить, начальник!')

20.19773057244549
19.115023759163584
18.240368239936576
Не могу делить, начальник!
26.096067251503698


ValueError: math domain error

In [48]:
x = [7, 8, 9, 0, 4, -5]

for item in x:
    try:
        print(42 / log(item + 1))
    except ZeroDivisionError:
        print('Не могу делить, начальник!')
    except ValueError:
        print('Не могу искать лоагрифм, начальник!')

20.19773057244549
19.115023759163584
18.240368239936576
Не могу делить, начальник!
26.096067251503698
Не могу искать лоагрифм, начальник!


In [49]:
x = 4
assert x < 5, 'На самом деле это не так'

In [50]:
x = 15
assert x < 5, 'На самом деле это не так'

AssertionError: На самом деле это не так

In [51]:
def f(x):
    if x > 5:
        raise ValueError('На вход пришли плохие данные, посмотри на них!')
    else:
        return x

In [52]:
f(3)

3

In [53]:
f(10)

ValueError: На вход пришли плохие данные, посмотри на них!

Можно отлавливать конкретные ошибки и записывать их в массив!

In [54]:
errors = [ ]

for x in [4, 5, 17, 22, 0, 5 , 8, 2]:
    try:
        print(f(x))
    except ValueError:
        errors.append(x)
        print('Возникла ошибка и я запомнил данные где она произошла')

4
5
Возникла ошибка и я запомнил данные где она произошла
Возникла ошибка и я запомнил данные где она произошла
0
5
Возникла ошибка и я запомнил данные где она произошла
2


In [55]:
errors

[17, 22, 8]

[Подробнее читайте в хэндбуке](https://academy.yandex.ru/handbook/python/article/model-isklyuchenij-python-try-except-else-finally-moduli)

## 3.3 Модуль itertools

Внутри этого модуля куча полезных итераторов.

Внутри набор быстрых и эффективных по памяти иснтументов. Вместе формируют "алгебру итераторов", которая позволяет лаконично и эффективно создвать на python разные штуки. Многие очень специфичные, но рано или поздно могут пригодиться. Рекомендую почитать что есть внутри.

In [91]:
from itertools import permutations

# Обещание сгенерировать все перестановки (генератор)
permutations([1, 2, 3])

In [92]:
list(permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [94]:
x = permutations(['Вика', 'Tom', 'Alex'])
list(x)

[('Вика', 'Tom', 'Alex'),
 ('Вика', 'Alex', 'Tom'),
 ('Tom', 'Вика', 'Alex'),
 ('Tom', 'Alex', 'Вика'),
 ('Alex', 'Вика', 'Tom'),
 ('Alex', 'Tom', 'Вика')]

In [95]:
list(x)

[]

In [96]:
# Генераторы отрабатывают только один раз!

Другое полезный пример - это groupby!

In [97]:
from itertools import groupby

student = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('daavee', 'B', 10),
    ]

По ключу `key` можно указать функцию, которая вычисляется при работе функции.

In [98]:
def f(w):
    return w[-1]
    
max(student, key=f)

('john', 'A', 15)

In [99]:
max(student, key=lambda w: w[-1])

('john', 'A', 15)

In [100]:
sorted(student, key=lambda w: w[-1])

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [101]:
sorted(student, key=lambda w: w[1])

[('john', 'A', 15), ('jane', 'B', 12), ('daavee', 'B', 10)]

In [102]:
groupby(student, key=lambda w: w[1])

In [103]:
list(groupby(student, key=lambda w: w[1]))

[('A', <itertools._grouper at 0x107c83490>),
 ('B', <itertools._grouper at 0x107c83be0>)]

In [104]:
[(i, list(v)) for i,v in groupby(student, key=lambda w: w[1]) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

__Важно:__ этот `groupby` глуповат и умеет группировать только отсротированные данные.

In [105]:
student = [
        ('jane', 'B', 12),
        ('john', 'A', 15),
        ('daavee', 'B', 10),
    ]

[(i, list(v)) for i,v in groupby(student, key=lambda w: w[1]) ]

[('B', [('jane', 'B', 12)]),
 ('A', [('john', 'A', 15)]),
 ('B', [('daavee', 'B', 10)])]

In [106]:
[(i, list(v)) for i,v in groupby(sorted(student, key=lambda w: w[1]), key=lambda w: w[1]) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

- [Документация](https://docs.python.org/3.9/library/itertools.html?highlight=itertools)
- [Обзорная статья на хабре с примерами кода](https://habr.com/ru/company/otus/blog/529356/)

## 3.4 Модуль operator

Внутри разные стандартные штуки вроде сложения, умножения и тп, но реализованные эффективно по памяти. Этот модуль очень удобно совмещать с itertools, код работает быстрее.

Одна из самых необычных штук - концепция геттеров. Это функция, которая извлекает по указанному пути нужные части структуры. Они полезны при работе с итераторами или генераторами последовательностей, так как вычисляются эффективнее, чем лямбда-функции, записанные в key.

In [108]:
from operator import itemgetter as at

student = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('daavee', 'B', 10),
    ]

# указал 2 координату
# lambda w: w[2]
sorted(student, key=at(2))

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [109]:
# указал 1 координату
sorted(student, key=at(1))

[('john', 'A', 15), ('jane', 'B', 12), ('daavee', 'B', 10)]

In [110]:
# сортируем сначала по оценке, а потом и по возрасту
sorted(student, key=at(2,1))

[('daavee', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [111]:
# распаковка groupby
[(i, list(v)) for i,v in groupby(student, key=at(1)) ]

[('A', [('john', 'A', 15)]), ('B', [('jane', 'B', 12), ('daavee', 'B', 10)])]

Можно заставлять его вытаскивать конкретные ключи, а не координаты!

In [113]:
dct = [{'a': 1}, {'a': 1}, {'a': 5}]

[(i, list(v)) for i,v in groupby(dct, at('a'))]

[(1, [{'a': 1}, {'a': 1}]), (5, [{'a': 5}])]

- [Документация](https://docs.python.org/3.9/library/operator.html)
- [Примеры использования, но сложноватые](https://pymotw.com/2/operator/)